Họ và tên: Phạm Đức Thể

MSSV: 19522253

Lớp: DS300.N11

LAB02 - 03/10/2022

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1ZGox04zOLURIu33GwlRGPPiE_0s39mNz?usp=sharing)

# **LAB 02 - THUẬT TOÁN LỌC CỘNG TÁC**

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import re
import time

from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from numpy.linalg import norm
from numpy import dot
from numpy import sqrt

sns.set()

In [ ]:
PATH = '/content/drive/MyDrive/TÀI LIỆU HỌC TẬP ĐẠI HỌC 2019-2023/NĂM 4 2022-2023/HỌC KỲ 1 2022 - 2023/DS300.N11 - HỆ KHUYẾN NGHỊ/THỰC HÀNH/LAB/LAB02 - 03 10 2022/dataset'

## [Movies rating dataset](https://grouplens.org/datasets/movielens/)

In [ ]:
# load dữ liệu từ file ratings
ratings = pd.read_csv(PATH+'/ml-latest-small/ratings.csv')
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [ ]:
# load dữ liệu từ file movies
movies = pd.read_csv(PATH+'/ml-latest-small/movies.csv')
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [ ]:
# load dữ liệu từ file tags
tags = pd.read_csv(PATH+'/ml-latest-small/tags.csv')
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [ ]:
# merge bảng ratings, và movies theo cột "movieID"
ratings_movies = pd.merge(ratings, movies, on='movieId')
ratings_movies

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...,...
100831,610,160341,2.5,1479545749,Bloodmoon (1997),Action|Thriller
100832,610,160527,4.5,1479544998,Sympathy for the Underdog (1971),Action|Crime|Drama
100833,610,160836,3.0,1493844794,Hazard (2005),Action|Drama|Thriller
100834,610,163937,3.5,1493848789,Blair Witch (2016),Horror|Thriller


In [ ]:
# Chuyển bảng ratings sang dataframe với các cột là "userId" và các hàng là "movieId" (*)
ratings_df = ratings.pivot(index='movieId', columns='userId')['rating'].fillna(0)
ratings_df

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Xây dựng thuật toán lọc cộng tác dựa trên User - Hệ số tương quan Pearson

In [ ]:
# Lấy ra rating của 1 user cho 1 movie
def get_rating(userid, movieid):
  return (ratings.loc[(ratings.userId==userid) & (ratings.movieId==movieid), 'rating'].iloc[0])

In [ ]:
get_rating(7,1)

4.5

In [ ]:
def pearson_correlation_score(user1, user2):
  both_watch_count = [] # Danh sách các bộ phim được xem chung bởi user1 và user2
  list_movie_user1 = ratings.loc[ratings.userId==user1, 'movieId'].to_list()
  list_movie_user2 = ratings.loc[ratings.userId==user2, 'movieId'].to_list()

  for movieid in list_movie_user1:
    if movieid in list_movie_user2:
      both_watch_count.append(movieid)
  
  if len(both_watch_count)==0:
    return 0
  
  sum_rating_1 = sum([get_rating(user1, i) for i in both_watch_count])
  avg_rating_1 = sum_rating_1/len(both_watch_count) # rating trung bình user1

  sum_rating_2 = sum([get_rating(user2, i) for i in both_watch_count])
  avg_rating_2 = sum_rating_2/len(both_watch_count) # rating trung bình user2

  TuSo = sum([(get_rating(user1, i) - avg_rating_1)*(get_rating(user2, i) - avg_rating_2) for i in both_watch_count])
  MauSo = sqrt(sum((get_rating(user1, i) - avg_rating_1)**2 for i in both_watch_count))*sqrt(sum((get_rating(user2, i) - avg_rating_2)**2 for i in both_watch_count))

  if MauSo==0:
    return 0
  
  return TuSo/MauSo

In [ ]:
pearson_correlation_score(50,200)

0.3960569995171616

### Xây dựng thuật toán lọc cộng tác dựa trên Item - Độ đo tương tự cosine

In [ ]:
def distance_similarity_score(user1, user2):
  both_watch_count = 0
  list_movie_user1 = ratings.loc[ratings.userId==user1, 'movieId'].tolist()
  list_movie_user2 = ratings.loc[ratings.userId==user2, 'movieId'].tolist()

  for movieid in list_movie_user1:
    if movieid in list_movie_user2:
      both_watch_count += 1
  
  if both_watch_count == 0:
    return 0

  rating1 = []
  rating2 = []

  for movieid in list_movie_user1:
    if movieid in list_movie_user2:
      rating1.append(get_rating(user1, movieid))
      rating2.append(get_rating(user2, movieid))
  
  # print(rating1)
  # print(rating2)

  return dot(rating1, rating2)/(norm(rating1)*norm(rating2))

In [ ]:
print('Distance based similarity between user ids X & u1: {}'.format(distance_similarity_score(1,2)))

Distance based similarity between user ids X & u1: 0.9999999999999998


In [ ]:
print('Distance based similarity between user ids X & u2: {}'.format(distance_similarity_score(1,3)))

Distance based similarity between user ids X & u2: 0.7919033104178548


In [ ]:
print('Distance based similarity between user ids X & u3: {}'.format(distance_similarity_score(1,4)))

Distance based similarity between user ids X & u3: 0.9328095891279707


### Xây dựng thuật toán lọc cộng tác dựa trên User với độ tương tự cosine
 Xuất ra top 10 bộ phim (chưa được người dùng xem) có dự đoán rating cao nhất cho UserID =20

In [ ]:
def most_similar_user(user1, number_of_user, similarity_name):
  userid = ratings.userId.unique().tolist()
  # print(len(userid))

  if similarity_name == 'pearson':
    similarity_score =  [(pearson_correlation_score(user1, user_i), user_i) for user_i in userid if user_i != user1]

  if similarity_name == 'cosine':
    similarity_score =  [(distance_similarity_score(user1, user_i), user_i) for user_i in userid if user_i != user1]

  similarity_score.sort() # Tăng dần
  similarity_score.reverse() # Giảm dần

  return similarity_score[:number_of_user] # Có thể thay đổi số lượng lân cận

In [ ]:
list_user_popular = most_similar_user(20, 10, "pearson")
list_user_popular

[(1.0000000000000002, 472),
 (1.0000000000000002, 378),
 (1.0000000000000002, 209),
 (1.0000000000000002, 127),
 (1.0000000000000002, 54),
 (1.0, 511),
 (1.0, 498),
 (1.0, 442),
 (1.0, 364),
 (1.0, 337)]

In [ ]:
# Danh sách các bộ phim đã rating bởi người dùng
def get_movieids(userid):
  return (ratings.loc[(ratings.userId==userid), 'movieId'].tolist())

In [ ]:
print(get_movieids(20))

[2, 8, 13, 34, 48, 107, 158, 239, 262, 317, 362, 364, 531, 551, 586, 588, 594, 595, 596, 661, 709, 720, 783, 801, 837, 899, 914, 919, 986, 1010, 1012, 1013, 1015, 1018, 1021, 1022, 1025, 1028, 1029, 1030, 1031, 1032, 1035, 1073, 1088, 1097, 1148, 1282, 1333, 1367, 1380, 1489, 1547, 1566, 1688, 1722, 1848, 1876, 1907, 1919, 1947, 2005, 2006, 2018, 2033, 2048, 2054, 2059, 2078, 2080, 2081, 2083, 2085, 2087, 2089, 2090, 2092, 2096, 2100, 2116, 2135, 2137, 2140, 2141, 2161, 2167, 2294, 2300, 2355, 2394, 2430, 2565, 2617, 2657, 2687, 2700, 2701, 2709, 2746, 2761, 2770, 2804, 2857, 2941, 2987, 3034, 3114, 3157, 3159, 3189, 3300, 3301, 3396, 3397, 3398, 3408, 3410, 3438, 3483, 3510, 3512, 3535, 3536, 3549, 3555, 3565, 3578, 3615, 3623, 3624, 3744, 3751, 3753, 3755, 3784, 3793, 3852, 3882, 3897, 3948, 3967, 3972, 3977, 3986, 3988, 3990, 3994, 3996, 3999, 4014, 4016, 4019, 4022, 4025, 4027, 4029, 4039, 4054, 4090, 4154, 4161, 4226, 4232, 4246, 4270, 4299, 4306, 4308, 4310, 4344, 4366, 4367, 436

In [ ]:
# Lấy ra tên của một bộ phim
def get_movie_title(movieid):
  return (movies.loc[(movies.movieId==movieid), 'title'].iloc[0])

In [ ]:
get_movie_title(500)

'Mrs. Doubtfire (1993)'

In [ ]:
# Lấy ra danh sách khuyến nghị từ top populars
def get_recommendation(userid, topN, similarity_name):
  total = {}
  sum_similarity = {}
  list_user_popular = most_similar_user(userid, topN, similarity_name)

  # Iterating over subset of user ids.
  for similarityName, user in list_user_popular:
    score = similarityName
    movieids = get_movieids(user) 
    for movieid in movieids: # -> Danh sách các id movie đã xem bởi user khác và khởi tạo giá trị = 0
      if movieid not in get_movieids(userid):
        total[movieid] = 0
        sum_similarity[movieid] = 0

    for movieid in movieids: #-> Danh sách các id movie đã xem bởi user khác
      if movieid not in get_movieids(userid):
        total[movieid] += get_rating(user, movieid)*score
        sum_similarity[movieid] += score

  # Normalizing ratings
  ranking = [(tot/sum_similarity[movieid], movieid) for movieid, tot in total.items()]
  ranking.sort()
  ranking.reverse()

  recommendations = [(get_movie_title(movieid), score) for score, movieid in ranking]
  return recommendations[:topN]

In [ ]:
get_recommendation(20, 10, 'cosine')

[('Project Almanac (2015)', 5.0),
 ('Deadpool (2016)', 5.0),
 ('Kingsman: The Secret Service (2015)', 5.0),
 ('Interstellar (2014)', 5.0),
 ('Conjuring, The (2013)', 5.0),
 ('Cabin in the Woods, The (2012)', 5.0),
 ('Avengers, The (2012)', 5.0),
 ('Tucker & Dale vs Evil (2010)', 5.0),
 ('Let Me In (2010)', 5.0),
 ('Up (2009)', 5.0)]

### Xây dựng thuật toán lọc cộng tác dựa trên Item với hệ số tương quan pearson
Xuất ra 10 người sẽ rating cao nhất (nghĩa là những người này chưa rating) cho bộ phim có movieId = 50

In [ ]:
# Chuyển bảng ratings sang dataframe với các cột là "movieId" và các hàng là "userId" (*)
Ratings_df = ratings.pivot(columns='movieId', index='userId')['rating'].fillna(0)
Ratings_df

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Lấy ra rating của 1 user cho 1 movie
def get_rating(userid, movieid):
  return (ratings.loc[(ratings.userId==userid) & (ratings.movieId==movieid), 'rating'].iloc[0])

In [ ]:
def pearson_correlation_score_new(movie1, movie2):
  both_watch_count = [] # Danh sách các bộ phim được xem chung bởi movie1 và movie2
  list_user_movie1 = ratings.loc[ratings.movieId==movie1, 'userId'].to_list()
  list_user_movie2 = ratings.loc[ratings.movieId==movie2, 'userId'].to_list()

  for userid in list_user_movie1:
    if userid in list_user_movie2:
      both_watch_count.append(userid)
  
  if len(both_watch_count)==0:
    return 0
  
  sum_rating_1 = sum([get_rating(i, movie1) for i in both_watch_count])
  avg_rating_1 = sum_rating_1/len(both_watch_count) # rating trung bình movie1

  sum_rating_2 = sum([get_rating(i, movie2) for i in both_watch_count])
  avg_rating_2 = sum_rating_2/len(both_watch_count) # rating trung bình movie2

  TuSo = sum([(get_rating(i, movie1) - avg_rating_1)*(get_rating(i, movie2) - avg_rating_2) for i in both_watch_count])
  MauSo = sqrt(sum((get_rating(i, movie1) - avg_rating_1)**2 for i in both_watch_count))*sqrt(sum((get_rating(i, movie2) - avg_rating_2)**2 for i in both_watch_count))

  if MauSo==0:
    return 0
  
  return TuSo/MauSo

In [ ]:
pearson_correlation_score_new(10,12)

0.7209286855927032

In [ ]:
def distance_similarity_score_new(movie1, movie2):
  both_watch_count = 0
  list_user_movie1 = ratings.loc[ratings.movieId==movie1, 'userId'].tolist()
  list_user_movie2 = ratings.loc[ratings.movieId==movie2, 'userId'].tolist()

  for userid in list_user_movie1:
    if userid in list_user_movie2:
      both_watch_count += 1
  
  if both_watch_count == 0:
    return 0

  rating1 = []
  rating2 = []

  for userid in list_user_movie1:
    if userid in list_user_movie2:
      rating1.append(get_rating(userid, movie1))
      rating2.append(get_rating(userid, movie1))
  
  # print(rating1)
  # print(rating2)

  return dot(rating1, rating2)/(norm(rating1)*norm(rating2))

In [ ]:
print('Distance based similarity between movie ids X & u1: {}'.format(distance_similarity_score_new(1,2)))

Distance based similarity between movie ids X & u1: 1.0000000000000002


In [ ]:
print('Distance based similarity between movie ids X & u2: {}'.format(distance_similarity_score_new(1,3)))

Distance based similarity between movie ids X & u2: 1.0


In [ ]:
print('Distance based similarity between movie ids X & u3: {}'.format(distance_similarity_score_new(1,4)))

Distance based similarity between movie ids X & u3: 0.9999999999999999


In [ ]:
def most_similar_movie(movie1, number_of_movie, similarity_name):
  movieid = ratings.movieId.unique().tolist()
  # print(len(movieid))

  if similarity_name == 'pearson':
    similarity_score =  [(pearson_correlation_score_new(movie1, movie_i), movie_i) for movie_i in movieid if movie_i != movie1]

  if similarity_name == 'cosine':
    similarity_score =  [(distance_similarity_score_new(movie1, movie_i), movie_i) for movie_i in movieid if movie_i != movie1]

  similarity_score.sort() # Tăng dần
  similarity_score.reverse() # Giảm dần

  return similarity_score[:number_of_movie] # Có thể thay đổi số lượng lân cận

In [ ]:
list_movie_popular = most_similar_movie(50, 10, "pearson")
list_movie_popular

[(1.0000000000000004, 156609),
 (1.0000000000000002, 184253),
 (1.0000000000000002, 166635),
 (1.0000000000000002, 118997),
 (1.0000000000000002, 109578),
 (1.0000000000000002, 96655),
 (1.0000000000000002, 96110),
 (1.0000000000000002, 93272),
 (1.0000000000000002, 91079),
 (1.0000000000000002, 90717)]

In [ ]:
# Danh sách các user đã rating cho bộ phim
def get_userids(movieid):
  return (ratings.loc[(ratings.movieId==movieid), 'userId'].tolist())

In [ ]:
print(get_userids(50))

[1, 5, 6, 7, 8, 16, 17, 18, 23, 24, 28, 29, 32, 33, 35, 38, 39, 41, 42, 45, 46, 54, 57, 58, 60, 61, 63, 64, 66, 68, 69, 72, 80, 84, 88, 91, 96, 103, 105, 109, 112, 117, 122, 124, 131, 132, 133, 140, 142, 145, 152, 156, 160, 162, 166, 174, 177, 178, 182, 187, 191, 195, 197, 198, 199, 202, 211, 215, 217, 219, 220, 221, 222, 228, 233, 237, 239, 241, 246, 247, 249, 254, 260, 261, 262, 266, 268, 274, 275, 278, 279, 280, 282, 287, 290, 294, 295, 296, 297, 298, 301, 305, 307, 308, 309, 313, 314, 317, 322, 323, 326, 327, 328, 329, 330, 331, 332, 335, 336, 337, 338, 343, 346, 348, 352, 354, 356, 357, 362, 363, 368, 370, 372, 373, 380, 381, 382, 385, 386, 387, 391, 394, 396, 400, 411, 414, 415, 418, 419, 424, 425, 432, 433, 435, 437, 444, 446, 448, 449, 453, 455, 462, 469, 470, 472, 474, 480, 482, 483, 495, 497, 501, 510, 512, 515, 522, 524, 525, 541, 553, 555, 560, 561, 562, 565, 566, 567, 568, 569, 573, 580, 587, 588, 590, 593, 595, 596, 597, 599, 601, 602, 606, 608, 610]


In [ ]:
# Lấy ra danh sách khuyến nghị từ top populars
def get_recommendation_new(movieid, topN, similarity_name):
  total = {}
  sum_similarity = {}
  list_movie_popular = most_similar_movie(movieid, topN, similarity_name)

  # Iterating over subset of movie ids.
  for similarityName, movie in list_movie_popular:
    score = similarityName
    userids = get_userids(movie)
    for userid in userids: # -> Danh sách các id user đã xem phim khác và khởi tạo giá trị = 0
      if userid not in get_userids(movieid):
        total[userid] = 0
        sum_similarity[userid] = 0

    for userid in userids: #-> Danh sách các id user đã xem phim khác
      if userid not in get_userids(movieid):
        total[userid] += get_rating(userid, movie)*score
        sum_similarity[userid] += score

  # Normalizing ratings
  ranking = [(tot/sum_similarity[userid], userid) for userid, tot in total.items()]
  ranking.sort()
  ranking.reverse()

  return ranking[:topN]

In [ ]:
get_recommendation_new(50, 10, 'pearson')

[(5.0, 586),
 (4.5, 564),
 (4.5, 89),
 (4.0, 563),
 (4.0, 556),
 (4.0, 184),
 (4.0, 125),
 (3.5, 534),
 (3.5, 365),
 (3.5, 73)]